In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.zip')
data.head(2)

user_id    basket_id  day  item_id  quantity  sales_value  store_id  \
0     2375  26984851472    1  1004906         1         1.39       364   
1     2375  26984851472    1  1033142         1         0.82       364   

   retail_disc  trans_time  week_no  coupon_disc  coupon_match_disc  
0         -0.6        1631        1          0.0                0.0  
1          0.0        1631        1          0.0                0.0

In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(10)

user_id                                             actual
0        1  [821867, 834484, 856942, 865456, 889248, 90795...
1        3  [835476, 851057, 872021, 878302, 879948, 90963...
2        6  [920308, 926804, 946489, 1006718, 1017061, 107...
3        7  [840386, 889774, 898068, 909714, 929067, 95347...
4        8  [835098, 872137, 910439, 924610, 992977, 10412...
5        9  [864335, 990865, 1029743, 9297474, 10457112, 8...
6       13  [6534178, 1104146, 829197, 840361, 862070, 884...
7       14  [840601, 867293, 933067, 951590, 952408, 96569...
8       15  [910439, 1082185, 959076, 1023958, 1082310, 13...
9       16                       [1062973, 1082185, 13007710]

In [6]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [7]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 136 ms


user_id                                             actual  \
0        1  [821867, 834484, 856942, 865456, 889248, 90795...   
1        3  [835476, 851057, 872021, 878302, 879948, 90963...   

                          popular_recommendation  
0  [6534178, 6533889, 1029743, 6534166, 1082185]  
1  [6534178, 6533889, 1029743, 6534166, 1082185]

In [8]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [15]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

Wall time: 4.08 s


user_id                                             actual  \
0        1  [  821867   834484   856942   865456   889248 ...   
1        3  [  835476   851057   872021   878302   879948 ...   

                         random_recommendation  \
0  [902280, 1040347, 8358234, 1026240, 966354]   
1   [9803278, 1011678, 876042, 918030, 910697]   

                          popular_recommendation  \
0  [6534178, 6533889, 1029743, 6534166, 1082185]   
1  [6534178, 6533889, 1029743, 6534166, 1082185]   

                                     itemitem  \
0  [999999, 1082185, 981760, 1127831, 995242]   
1  [999999, 1082185, 981760, 1098066, 995242]   

                                        cosine  \
0  [1082185, 999999, 981760, 1127831, 1098066]   
1   [1082185, 1098066, 981760, 999999, 826249]   

                                         tfidf  \
0  [1082185, 981760, 1127831, 999999, 1098066]   
1   [1082185, 981760, 1098066, 826249, 999999]   

                                  own_purchases  
0   [999999, 1082185, 1029743, 995785, 1004906]  
1  [999999, 1082185, 1098066, 6534178, 1127831]

In [16]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

item_id  n_sold
0    25671       6
1    26081       1
2    26093       1
3    26190       1
4    26355       2

In [18]:
popularity.shape

(86865, 2)

In [20]:
all_items_sorted_by_polularity = popularity.sort_values('n_sold', ascending=False).item_id.tolist()
all_items_sorted_by_polularity

[6534178,
 6533889,
 6534166,
 6544236,
 1404121,
 397896,
 1426702,
 5703832,
 480014,
 5668996,
 6410462,
 420647,
 5747420,
 731106,
 5716076,
 707683,
 202291,
 1153346,
 1082185,
 5747233,
 6410464,
 545926,
 995242,
 2848087,
 1388206,
 1029743,
 5712216,
 5850988,
 1133018,
 1106523,
 1007195,
 981760,
 5845857,
 883404,
 1127831,
 2690723,
 866227,
 995785,
 860776,
 951590,
 5569230,
 908531,
 916122,
 826249,
 1098066,
 862349,
 1058997,
 1044078,
 904360,
 840361,
 923746,
 1126899,
 849843,
 961554,
 1005186,
 820165,
 1053690,
 844179,
 844165,
 1070820,
 1065593,
 834484,
 994928,
 833715,
 859075,
 1022003,
 1013321,
 938700,
 854852,
 986947,
 5569471,
 927191,
 1071939,
 1096036,
 1004906,
 986912,
 1080414,
 914190,
 908846,
 962229,
 8090521,
 1085604,
 911878,
 1092026,
 866211,
 1068719,
 878996,
 8090537,
 1081177,
 847270,
 929668,
 1024306,
 909894,
 907014,
 903325,
 910032,
 1095275,
 833025,
 862139,
 962568,
 953476,
 847982,
 999971,
 968215,
 1028816,
 976

In [ ]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train_top5000 = data_train.copy().loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999


user_item_matrix_top5000 = pd.pivot_table(data_train_top5000, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix_top5000[user_item_matrix_top5000 > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

# Оценивание
За выполнени каждого задания 1 балл

4 балла -> отл

3 балла -> хор

И тд

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - это товар, который купили пользователи, если они купиши товар из top5000. Используя этот товар мы смещяем качество рекомендаций. В какую сторону? Уберите этот товар и сравните с качеством на семинаре.

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [10]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
        
    items = np.array(item_weight['item_id'])
    weights = np.array(item_weight['weight'])
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

Сделайте предсказания

In [11]:
%%time

# your_code

Wall time: 0 ns


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [12]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

user_id                                             actual  \
0        1  [  821867   834484   856942   865456   889248 ...   
1        3  [  835476   851057   872021   878302   879948 ...   

                         random_recommendation  \
0  [1039767, 9705367, 830271, 2716227, 926763]   
1  [832140, 1077771, 895263, 6424469, 1399822]   

                          popular_recommendation  \
0  [6534178, 6533889, 1029743, 6534166, 1082185]   
1  [6534178, 6533889, 1029743, 6534166, 1082185]   

                                     itemitem  \
0  [999999, 1082185, 981760, 1127831, 995242]   
1  [999999, 1082185, 981760, 1098066, 995242]   

                                        cosine  \
0  [1082185, 999999, 981760, 1127831, 1098066]   
1   [1082185, 1098066, 981760, 999999, 826249]   

                                         tfidf  \
0  [1082185, 981760, 1127831, 999999, 1098066]   
1   [1082185, 981760, 1098066, 826249, 999999]   

                                  own_purchases  
0   [999999, 1082185, 1029743, 995785, 1004906]  
1  [999999, 1082185, 1098066, 6534178, 1127831]

In [13]:
# your_code

### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
-  *Попробуйте стратегии ансамблирования изученных алгоритмов

Обязательно нужно сделать первые 2 пункта!

In [14]:
# your_code

### Задание 4. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.